# Strategy

- S&P 500 (SPY) vs Intermediate-term US Treasuries (IEF)
- Gold (GLD) vs Long-term US Treasuries (TLT)
- US Real Estate (VNQ) vs Intermediate-term US Treasuries (IEF)


![strategy_params](strategy_params.png)


[More info](https://allocatesmartly.com/stokens-active-combined-asset-strategy/)

IEF - 7-10 years bonds

TLT - 20+ bonds


In [ ]:
CONFIG = {
  'SPY': {'upper': 126, 'lower': 252, 'defensive_asset': 'IEF'},
  'GLD': {'upper': 252, 'lower': 126, 'defensive_asset': 'TLT'},
  'VNQ': {'upper': 126, 'lower': 252, 'defensive_asset': 'IEF'}
}

In [ ]:
import numpy as np

ASSETS = set(np.array([
  [asset, config['defensive_asset']] 
    for (asset, config) in CONFIG.items()
]).flat)

print(f'All assets used: {ASSETS}')

# Load assets history

We will use yahoo finance client to load dividends and prices history for our `ASSETS`.

In [ ]:
# pip install --user pandas numpy datetime scipy pandas_datareader yfinance

import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from datetime import datetime, date, timedelta

yf.pdr_override()

In [ ]:
def load_max_history(assets):
  df = None

  for asset in assets:
    data = yf.Ticker(asset).history(period='max')
    columns = pd.MultiIndex.from_product(
      [data.columns, [asset]],
      names=['property', 'asset']
    )
    multi_level_df = pd.DataFrame(data.values, index=data.index, columns=columns)
    # display(multi_level_df.columns.to_numpy())
    if df is None:
      df = multi_level_df
    else:
      df = df.join(multi_level_df)

  df = df[sorted(df.columns)]
  return df.dropna()

df = load_max_history(ASSETS)[['Close', 'Dividends']]
df.head()

# Stategy evaluaton

Create upper and lower channels for risk-on assets defined in the `CONFIG`.

In [ ]:
from operator import itemgetter

In [ ]:
for (risk_asset, config) in CONFIG.items():
  (upper_channel_window, lower_channel_window) = itemgetter('upper', 'lower')(config)
  close_prices = df.Close[risk_asset]

  df['Upper_Channel', risk_asset] = close_prices\
    .rolling(window=upper_channel_window).max().shift(1)
  df['Lower_Channel', risk_asset] = close_prices\
    .rolling(window=lower_channel_window).min().shift(1)

df = df[sorted(df.columns)]

In [ ]:
df.iloc[250:].head() # deliberate shift to check the first moment where 252 lower/upper channel appeared for the first time

In [ ]:
df.iloc[124:].head() # deliberate shift to check the first moment where 126 lower/upper channel appeared for the first time

In [ ]:
from dotmap import DotMap
import rpar

SLIPPAGE = 0.01
FEE = 0.02
TAX = 0.15 # we pay 15% tax on dividends here (but it can vary a lot depending on circumstances)

cash = 50000 # initial cash
portfolio = []

def flatten_portfolio(portfolio): 
  dict = {}
  for item in portfolio:
    dict[item['asset']] = item['amount']
  return dict

results = DotMap(portfolio_value=[], cash=[], portfolio=[], last_fractions=[])
# TODO add turnaround
total = DotMap(fees=0, volume=0)

# RISK_BALANCE_WINDOW = float('inf')
RISK_BALANCE_WINDOW = 2 * 256

# history = df.iloc[0:5]
history = df
for x in range(0, len(history)):

  today = history.iloc[x]
  close_price = today.Close
  dividends = today.Dividends

  fractions = {}

  # initialize portfolio - start with risk on
  if len(portfolio) == 0:
    portfolio = [{}] * len(CONFIG)
    assets = CONFIG.keys()
    portfolio_value = cash
    fractions = dict(zip(assets, [1 / len(assets)] * len(assets)))

    for (index, (asset, config)) in enumerate(CONFIG.items()):
      portfolio[index] = {'asset': asset, 'amount': 0}
      # we will open positions later during the rebalance phase

  # add dividends
  for item in portfolio:
    (asset, amount) = itemgetter('asset', 'amount')(item)
    maybe_divs = dividends[asset]
    if not np.isnan(maybe_divs):
      cash += (amount * maybe_divs) * (1 - TAX) 

  # check if need to replace some assets with defensive assets
  upper_channel = today.Upper_Channel
  lower_channel = today.Lower_Channel

  new_assets = []

  for (index, (risk_asset, config)) in enumerate(CONFIG.items()):
    my_asset = portfolio[index]['asset']
    if not pd.isna(upper_channel[risk_asset]) and close_price[risk_asset] > upper_channel[risk_asset]:
      new_assets.append(risk_asset)
    elif not pd.isna(lower_channel[risk_asset]) and close_price[risk_asset] < lower_channel[risk_asset]:
      new_assets.append(config['defensive_asset'])
    else:
      new_assets.append(my_asset)

  needs_rotation = any(new_asset != item['asset'] for (new_asset, item) in zip(new_assets, portfolio))
  if needs_rotation:
    # maybe update risk balanced weights based on the previous RISK_BALANCE_WINDOW prices
    # note: if RISK_BALANCE_WINDOW = float('inf') this will never happen in this case - always use equal weight
    fractions = dict(zip(new_assets, [1 / len(new_assets)] * len(new_assets)))
    if x > RISK_BALANCE_WINDOW:
      fractions = dict(rpar.get_weights(df.iloc[x - RISK_BALANCE_WINDOW:x].Close[new_assets]))
      # this protects from cases when a single asset is repeated twice e.g. IEF/GLD/IEF state
      for (k, v) in fractions.items():
        if isinstance(fractions[k], pd.Series): 
          fractions[k] = v[0]

    for (index, item) in enumerate(portfolio):
      # do we need to replace the item (i.e. switch risk/defensive)
      if new_assets[index] != item['asset']:
        # sell current asset, we will buy new asset later during rebalance
        (asset, amount) = itemgetter('asset', 'amount')(item)
        cash += amount * (close_price[asset] - SLIPPAGE)

        fee = amount * FEE
        cash -= fee
        total.fee += fee
        total.volume += amount

        item['asset'] = new_assets[index] 
        item['amount'] = 0

  # calculate current portfolio value
  portfolio_value = cash
  for item in portfolio:
    (asset, amount) = itemgetter('asset', 'amount')(item)
    portfolio_value += (close_price[asset] - SLIPPAGE - FEE) * amount

  if len(fractions) == 0: 
    fractions = results.last_fractions[-1] 

  for item in portfolio:
    asset = item['asset']
    price = close_price[asset]
    new_amount = np.floor(portfolio_value * fractions[asset] / price)
    diff = item['amount'] - new_amount
    if diff != 0:
      # rebalance
      if diff > 0: # => item['amount'] > new_amount => sell
        cash += abs(diff) * (price - SLIPPAGE)
      else: # => item['amount'] < new_amount => buy
        cash -= abs(diff) * (price + SLIPPAGE)

      fee = abs(diff) * FEE
      cash -= fee
      total.fee += fee
      total.volume += abs(diff)

      item['amount'] = new_amount

  # calculate updated portfolio value
  portfolio_value = cash
  for item in portfolio:
    (asset, amount) = itemgetter('asset', 'amount')(item)
    portfolio_value += (close_price[asset] - SLIPPAGE - FEE) * amount

  results.portfolio_value.append(portfolio_value)
  results.cash.append(cash)
  results.portfolio.append(flatten_portfolio(portfolio))
  results.last_fractions.append(fractions)

result_df = pd.DataFrame(results, index=history.index)

# dumping results dataframe to HTML can be good for debug purposes
#
# with open('out.html', 'w') as file:
#   file.write(history.join(result_df).to_html())


## Portfolio states

In [ ]:
result_df.head()

In [ ]:
result_df.tail()

# Total annualized returns

In [ ]:
first = result_df.iloc[0]
last = result_df.iloc[-1]

years = (last.name - first.name).days / 365.25

annualized_return = (last['portfolio_value']  / first['portfolio_value']) ** (1/years) - 1

print(
  """Annualized return is: %.2f%%
over the period of %.2f years
from %s to %s""" % (
    100 * annualized_return,
    years,
    date.fromtimestamp(first.name.timestamp()), 
    date.fromtimestamp(last.name.timestamp())
  )
)


# Overall performance

In [ ]:
print(f'Fees paid {total.fee:.2f}')
print(f'Volume traded {total.volume:.2f} stocks')

## Free cash

In [ ]:
result_df.cash.plot(figsize=(15, 8))

## Equity

In [ ]:
log_returns = np.log(result_df.portfolio_value).diff()

cum_returns = log_returns.cumsum() 

CAPITAL = 10000

perf = CAPITAL * np.exp(cum_returns)
plt.figure()
perf.plot(figsize=(15,8), color='b', alpha=0.7)

print(f'Initial investment at {perf.index[0].date()}: {CAPITAL}')
print(f'Last value at {perf.index[-1].date()}: {perf[-1]:.2f}')

In [ ]:
TRADING_DAYS_YEARLY = 252
# RISK_FREE_RATE = 0.02
RISK_FREE_RATE = 0.00

annual_return = np.exp(log_returns.mean() * TRADING_DAYS_YEARLY) - 1
annual_volatility = log_returns.std() * np.sqrt(TRADING_DAYS_YEARLY)

sharpe_ratio = (annual_return - RISK_FREE_RATE) / annual_volatility

annualized_downside = log_returns.loc[log_returns<0].std() * np.sqrt(TRADING_DAYS_YEARLY)
sortino_ratio = (annual_return - RISK_FREE_RATE) / annualized_downside  

sortino_ratio = (annual_return - RISK_FREE_RATE) / annualized_downside  

print(
  f"""Annulaized return {(annual_return * 100):.2f}%
Annualized volatility {(annual_volatility * 100):.2f}%
Sharpe ratio {sharpe_ratio:.2f}
Sortino ratio {sortino_ratio:.2f}
""" 
)

# Returns by month/by year

In [ ]:

def montly_results(result_df):
  initial_value = result_df['portfolio_value'][0]

  grouping = pd.MultiIndex.from_tuples(
    zip(result_df.index.year, result_df.index.month),
    names=['year', 'month']
  )

  by_month = result_df.groupby(by=grouping).last()[['portfolio_value']]
  by_month = by_month.join(by_month.shift(1).add_prefix('prev_')).fillna(initial_value)

  table = pd.DataFrame(
    {
      'change': 100 * (by_month['portfolio_value'] / by_month['prev_portfolio_value'] - 1),
    },
    index = grouping
  ).dropna().reset_index()

  montly_results = table.pivot_table(index=['year'], columns=['month'], values=['change'])

  grouping = result_df.index.year
  by_year = result_df.groupby(by=grouping).last()[['portfolio_value']]
  by_year = by_year.join(by_year.shift(1).add_prefix('prev_')).fillna(initial_value)

  montly_results['total'] = 100 * (by_year['portfolio_value'] / by_year['prev_portfolio_value'] - 1)

  return montly_results

pd.options.display.float_format = lambda x: 'N/A' if np.isnan(x) else '{:,.2f}%'.format(x)
display(montly_results(result_df))
pd.options.display.float_format = None 


# Drawdown analysis

In [ ]:
last_peak = cum_returns.cummax()

log_dd = cum_returns - last_peak

pct_dd = (np.exp(log_dd) - 1) * 100

ax = pct_dd.plot(figsize=(15,8), color='r', alpha=0.7)
ax.grid(axis='both')
ax.set_ylabel('Drawdown, %')

# plt.savefig('drawdown_evaluation.svg')


## Longest/max drawdowns

In [ ]:

def find_true_streaks(array):
  # finds all "True" streaks in array from longest to shortest

  masked = np.concatenate(([False], array, [False]))       
  true_streaks = np.flatnonzero(masked[1:] != masked[:-1]).reshape(-1, 2) 
  ends = true_streaks[:,1]
  true_streaks_descending = (true_streaks[:,1] - true_streaks[:,0]).argsort()[::-1]
  return true_streaks[true_streaks_descending]

dd_streaks = find_true_streaks(pct_dd.values != 0)

def get_streak(n):
  start, end = dd_streaks[n]
  end = end - 1
  days = (pct_dd.index[end] - pct_dd.index[start]).days
  years = days / 365.25
  months = years * 12
  max_dd_index = start + np.argmin(pct_dd.iloc[start:end])
  max_depth = pct_dd.iloc[max_dd_index]

  return DotMap(
    rank=n+1,
    days=days,
    months=months,
    years=years,
    from_date=pct_dd.index[start].date(),
    to_date=pct_dd.index[end].date(),
    max_depth=max_depth,
    max_dd_date=pct_dd.index[max_dd_index].date(),
  )

def print_streak(n):
  streak = get_streak(n)
  print(
    f"""#{streak.rank}: Drawdown: {streak.days} days = {streak.months:.2f} months = {streak.years:.2f} years
From {streak.from_date} to {streak.to_date}""" 
  )

streaks_info = []
for i in range(12):
  if dd_streaks.size <= i: break
  streaks_info.append(get_streak(i).values())

if dd_streaks.size == 0:
  print("No drawdown found")
else:
  dd_df = pd.DataFrame(streaks_info, columns=get_streak(0).keys()).set_index(['rank'])
  pd.options.display.float_format = '{:,.2f}'.format
  print('Longest drawdowns')
  display(dd_df)
  print('By depth')
  display(dd_df.sort_values(by=['max_depth']))
  pd.options.display.float_format = None 

print()
max_dd_index = np.argmin(pct_dd)
print(f"""Absolute max drawdown {pct_dd.iloc[max_dd_index]:.2f}% on {pct_dd.index[max_dd_index].date()}""")

## Drawdown quantiles

In [ ]:
from scipy.stats import expon

nonzero_dd_days = -pct_dd[pct_dd < 0].sort_values()
size = nonzero_dd_days.size

nonzero_dd_days.hist(bins=32, alpha=0.5, density=True, color='red')

params = expon.fit(nonzero_dd_days)

def quantile(p):
  return expon.ppf(p, *params)

xs = np.linspace(quantile(0.001), quantile(0.999), 100)
plt.plot(xs, expon.pdf(xs, *params))

def print_quantile(p):
  theoretical = quantile(p)
  real = nonzero_dd_days[int(size * (1 - p))]
  print(f'{p * 100}% of time expected drawdown is no more than {theoretical:.2f}% theoretically or {real:.2f}% actually')

print('Quantiles:')
for p in [0.50, 0.75, 0.95, 0.99]:
  print_quantile(p)

print(f'Max DD was {nonzero_dd_days[0]:.2f}%')
print(f'Last DD ({pct_dd.index[-1].date()}) was {-pct_dd[-1]:.2f}%')

# Current portfolio state

## Structure

In [ ]:
last_prices = df.iloc[-1].Close
current_portfolio = result_df.iloc[-1].portfolio
total_value = 0
for asset in current_portfolio.keys():
  total_value += last_prices[asset] * current_portfolio[asset]

for asset in current_portfolio.keys():
  print(f'{asset}: {(last_prices[asset] * current_portfolio[asset]) / total_value:.2f}%')

## Channels

In [ ]:
last_day = df.iloc[-1]

lower = []
upper = []
current = []
defensive = []
index = []
for (risk_asset, config) in CONFIG.items():
  index.append(risk_asset)
  lower.append(last_day.Lower_Channel[risk_asset])
  current.append(last_day.Close[risk_asset])
  upper.append(last_day.Upper_Channel[risk_asset])
  defensive.append(CONFIG[risk_asset]['defensive_asset'])

status_df = pd.DataFrame({'Lower': lower, 'Current price': current, 'Upper': upper, 'Defensive asset': defensive}, index=pd.Index(index, name='Risk asset'))
pd.options.display.float_format = '${:,.2f}'.format
current_assets = list(current_portfolio.keys())
display(
  status_df.reset_index().style.apply(
    lambda col:  [
      'background: lime' if current_assets[i] == asset else ''
      for (i, asset) in enumerate(col)
    ] if col.name in ['Risk asset', 'Defensive asset'] else [''] * len(col), axis = 0
  )
)
pd.options.display.float_format = None


